# CS 412 HW01
Xiaoyun Zhuang
2/24/2021

In [1]:
import numpy as np
import pandas as pd
from scipy import stats

## Question 1
`pd.describe()` is used to find the man, quartiles, mean and variances\
`pd.mode()` is used to find the mode.
### 1.(a) Maximum and minimum
Midterm socres: 37, 100
Final scores: 35, 100

### 1.(b) First quartile Q1, median, and third quartile Q3
Midterm socres: 68, 77, 87\
Final scores: 82, 89, 96

### 1.(c) Mean
Midterm scores: 76.715\
Final scores: 87.084

### 1.(d) Mode
Midterm scores: 77, 83\
Final scores: 97

### 1.(e) Variance
Midterm scores: 173.279\
Final scores: 119.232

In [2]:
scores = pd.read_csv("data.online.scores.txt", delimiter="\t", names=["ID","Midterm","Final"])
print(scores.describe())
print("The mode of midterm is: " + ' '.join(map(str, scores.Midterm.mode()[0:].to_list())))
print("The mode of final is: " + ' '.join(map(str, scores.Final.mode()[0:].to_list())))
print("The variance of midterm is: " + str(13.16355**2))
print("The variance of final is: " + str(10.919349**2))


                ID     Midterm        Final
count  1000.000000  1000.00000  1000.000000
mean    499.500000    76.71500    87.084000
std     288.819436    13.16355    10.919349
min       0.000000    37.00000    35.000000
25%     249.750000    68.00000    82.000000
50%     499.500000    77.00000    89.000000
75%     749.250000    87.00000    96.000000
max     999.000000   100.00000   100.000000
The mode of midterm is: 77 83
The mode of final is: 97
The variance of midterm is: 173.2790486025
The variance of final is: 119.232182583801


## Question 2

- First, use `zscore` in `scipy` package to calculate the normalized scores.
- Then, use the `.var()`, `.corr()`, and `.cov()` in `pandas` to calculate the variance, correlation coefficient and covariance.

### 2.(a) Compute and compare the variance of midterm-original and midterm-normalized
Midterm_original：173.279\
Midterm_normalized: 1.001

### 2.(b) Given an original midterm score of 90, what is the corresponding score after normalization?
1.009

### 2.(c) Compute the Pearson’s correlation coefficient between midterm-original and finals-original.
0.544425

### 2.(d) Compute the Pearson’s correlation coefficient between midterm-normalized and finals-original.
0.544425

### 2.(e)  Compute the covariance between midterm-original and finals-original.
78.2541941941942

In [3]:
from scipy.stats import zscore
scores = pd.read_csv("data.online.scores.txt", delimiter="\t", names=["ID","Midterm","Final"])
scores_z = scores.apply(zscore)
data = [scores["Midterm"], scores_z["Midterm"]]
headers = ["midterm_original", "midterm_normalized"]
mid = pd.concat(data, axis=1, keys=headers)
print("The variance of the midterm scores are as below:")
print(mid.var())

The variance of the midterm scores are as below:
midterm_original      173.279054
midterm_normalized      1.001001
dtype: float64


In [4]:
norm_90 = mid.loc[mid["midterm_original"]==90, "midterm_normalized"].iloc[1]
print("The normalized score for 90 is: " + str(norm_90))

The normalized score for 90 is: 1.009731210573523


In [5]:
cor1 = scores.corr(method="pearson").iloc[1,2]
print("The Pearson’s correlation coefficient between midterm-original and finals-original is: "+str(cor1))

The Pearson’s correlation coefficient between midterm-original and finals-original is: 0.5444247423124133


In [6]:
cor2 = pd.concat([scores_z["Midterm"], scores["Final"]], axis=1, keys=["midterm_normalized", "finals_original"]).corr(method="pearson").iloc[0,1]
print("The Pearson’s correlation coefficient between midterm-normalized and finals-original is: "+str(cor2))

The Pearson’s correlation coefficient between midterm-normalized and finals-original is: 0.544424742312412


In [7]:
cov = scores.cov().iloc[1,2]
print("Tthe covariance between midterm-original and finals-original is: "+str(cov))

Tthe covariance between midterm-original and finals-original is: 78.2541941941942


## Question 3

- The original data is given as string, so first we need to change the format to format.
- `distance.minkowski()` and `distance.cosine()` are used to calculate the Minkowski distance and cosine distance. As we need the cosine similarity, we need to subsract the cosince distance derived from 1.
- To calculate KL divergence, we need to first construct probability distribution and then use `scipy.special.rel_entr` function.


### 3.(a) compute the Minkowski distance of the vectors for CML and CBL with regard to different `h` values:
`h=1:`6152\
`h=2:`715.328\
`h=3:`170

### 3.(b) Compute the cosine similarity between the feature vectors for CML and CBL
0.841

### 3.(c) Compute the Kullback-Leibler (KL) divergence between CML and CBL by constructing probability distributions for each library based on their feature vectors.
0.207

In [8]:
df = pd.read_csv("data.libraries.inventories.txt", delimiter="\t",header=None)
df = df.T
new_header = df.iloc[0]
df = df[1:]
df.columns = new_header
df[["CML", "CBL"]] = df[["CML", "CBL"]].apply(pd.to_numeric)
df.head()
cml = df["CML"].to_numpy()
cbl = df["CBL"].to_numpy()

In [9]:
from scipy.spatial import distance
print("Minkowski distance of the vectors for CML and CBL with regard to different h values are:")
print(distance.minkowski(cml, cbl, 1))
print(distance.minkowski(cml, cbl, 2))
print(distance.minkowski(cml, cbl, float("inf")))

Minkowski distance of the vectors for CML and CBL with regard to different h values are:
6152.0
715.3278968417211
170.0


In [10]:
print("The cosine similarity between the feature vectors for CML and CBL is: ")
print(1-distance.cosine(cml, cbl))

The cosine similarity between the feature vectors for CML and CBL is: 
0.8414040256623079


In [11]:
from scipy.special import rel_entr
cml_dist=(df["CML"]/sum(df["CML"])).to_numpy()
cbl_dist=(df["CBL"]/sum(df["CBL"])).to_numpy()
print("The KL divergence of the distributions for the libraries is:")
print(sum(rel_entr(cml_dist, cbl_dist)))

The KL divergence of the distributions for the libraries is:
0.20708093733159474


## Question 4

- First create a numpy array from the table
- Given the symmetric binary variables, the distance can be derived from $\frac{r+s}{q+s+r+t}$ 
- `stat.chi2_contingency()` gives me a quick way to calculate the $\chi^2$ statistic
- Also, `stat.chi2_contingency()` function will provide us the p-value, degree of freedom as well. By comparing the p-value with the $\alpha$, we can check if we need to reject the null hypothesis

### 4.(a) The distance between the binary attributes Buy Beer and Buy Diaper by assuming they are symmetric binary variables.
0.0157

### 4.(b) The Jaccard coefficient between Buy Beer and Buy Diaper.
0.7317

### 4.(c) The $\chi^2$ statistic for the contingency table.
2450.716

### 4.(d) Consider a hypothesis test based on the $\chi^2$ statistic where the null hypothesis is that Buy Beer and Buy Diaper are independent. Can you reject the null hypothesis at a significance level of α = 0.05? Explain your answer, and also mention the degrees of freedom used for the hypothesis test.
We have to reject the null hypothesis as the p-value derived is smaller than 0.05.


In [14]:
cust = np.array([[150, 40], [15, 3300]])
beer = cust[0][1]
diaper = cust[1][0]
print("The distance between the binary attributes Buy Beer and Buy Diaper is:", (beer+diaper)/cust.sum())

The distance between the binary attributes Buy Beer and Buy Diaper is: 0.015691868758915834


In [15]:
jac = cust[0][0] / (cust[0][0]+cust[0][1]+cust[1][0])
print("The jaccard coefficient of between Buy Beer and Buy Diaper is:", jac)

The jaccard coefficient of between Buy Beer and Buy Diaper is: 0.7317073170731707


In [16]:
print("The chi=square statistic for the contingeny table is:", stats.chi2_contingency(cust)[0])

The chi=square statistic for the contingeny table is: 2450.716326822006


In [17]:
if(stats.chi2_contingency(cust)[1]<0.05):
    print("We have to reject the null hypothesis given the p-value as", 
          stats.chi2_contingency(cust)[1], 
          "and the degree of freedom of", stats.chi2_contingency(cust)[2])
else:
    print("We are not able to reject the null hypothesis")

We have to reject the null hypothesis given the p-value as 0.0 and the degree of freedom of 1
